# Elixir: I/O and the File System

### [IO module](https://hexdocs.pm/elixir/IO.html)
* The IO module is the main mechanism for reading and writing to standard input/output (:stdio), standard error (:stderr), files, and other IO devices.

In [1]:
IO.puts "hello world"

hello world


:ok

![gets](pix/gets.png)

* By default, IO functions read from standard input and write to the standard output. We can change that by passing, for example, :stderr as an argument (in order to write to the standard error device):

In [3]:
IO.puts :stderr, "hello world"

hello world


:ok

### [File module](https://hexdocs.pm/elixir/File.html)
* The File module allows us to open files as IO devices. By default, files are opened in binary mode, which requires developers to use __IO.binread__ and __IO.binwrite/2__.

In [4]:
{:ok, file} = File.open "hello", [:write]

{:ok, #PID<0.232.0>}

In [5]:
IO.binwrite file, "world"

:ok

In [6]:
File.close file

:ok

In [7]:
File.read "hello"

{:ok, "world"}

* A file can be opened with :utf8 encoding, which tells the File module to interpret the bytes read from the file as UTF-8-encoded bytes.

* The File module has many functions, named after their UNIX equivalents, to work with the file system. 
   * File.rm: remove files
   * File.mkdir: create directories
   * File.mkdir_p: create directories and all their parent chain
   * File.cp_r and File.rm_rf: copy and remove files and directories recursively
   
* File functions have two variants: one “regular” variant and another variant with a trailing bang (!). For example, when we read the "hello" file in the example above, we use File.read. Alternatively, we can use File.read!:

In [8]:
File.read "hello"

{:ok, "world"}

In [9]:
File.read! "hello"

"world"

In [10]:
File.read "unknown"

{:error, :enoent}

In [11]:
File.read! "unknown"

File.Error: 1

* The version with ! returns the contents of the file instead of a tuple, and if anything goes wrong the function raises an error.
* The version without ! is preferred when you want to handle different outcomes using pattern matching:

### [Path module](https://hexdocs.pm/elixir/Path.html)
* Most File functions expect paths as arguments. Most commonly, those paths will be regular binaries. The Path module provides facilities for working with such paths.

In [11]:
Path.join("foo", "bar")

"foo/bar"

In [12]:
Path.expand("~/hello")

"/home/bjpcjp/hello"

* Using Path functions is preferred since the Path module takes care of different operating systems transparently. Keep in mind that Elixir will automatically convert slashes (/) into backslashes (\) on Windows when performing file operations.

### Processes and group leaders
* Recall that File.open returns a tuple like {:ok, pid}:

In [13]:
{:ok, file} = File.open "hello", [:write]

{:ok, #PID<0.288.0>}

* This happens because the IO module actually works with processes. When you write IO.write(pid, binary), the IO module will send a message to the process identified by pid with the desired operation.
* The StringIO module implements IO device messages on top of strings:

In [15]:
{:ok, pid} = StringIO.open("hello")

{:ok, #PID<0.301.0>}

In [16]:
IO.read(pid,2)

"he"

* By modeling IO devices with processes, the Erlang VM allows different nodes in the same network to exchange file processes in order to read/write files in between nodes. Of all IO devices, there is one that is special to each process: __the group leader__.
* When you write to :stdio, you are actually sending a message to the group leader, which writes to the standard-output file descriptor.
* Group leaders can be configured per process. For example, when executing code in a remote terminal, a group leader can guarantee messages in a remote node are redirected and printed in the terminal that triggered the request.

In [17]:
IO.puts :stdio, "hello"

hello


:ok

In [18]:
IO.puts Process.group_leader, "hello"

hello


:ok

### iodata and chardata
* In the examples above, we used binaries when writing to files. Recall that strings are made of bytes while charlists are lists with Unicode codepoints.
* The functions in IO and File accept list arguments. They also accept mixed lists of lists, integers, and binaries.

In [19]:
IO.puts 'hello world'

hello world


:ok

In [20]:
IO.puts ['hello', ?\s, "world"]

hello world


:ok

* Using lists in IO operations requires some attention. A list may represent either a bunch of bytes or characters - which one to use depends on the encoding of the IO device. 
* If the file is opened without encoding, the file is expected to be in raw mode, and the functions in the IO module starting with bin* must be used. 
* Those functions expect an iodata as an argument; i.e., they expect a list of integers representing bytes or binaries to be given.

* :stdio and files opened with :utf8 encoding work with the remaining functions in the IO module. Those functions expect a char_data as an argument (a list of characters or strings).

* Although this is a subtle difference, you only need to worry about these details if you intend to pass lists to those functions. Binaries are already represented by the underlying bytes and as such their representation is always “raw”.